## Pulling Option Chains from Finance Yahoo

## Installation Steps

This parser depends on the following libraries being installed:
- yahoo_fin
- mysql

In order to install yahoo_fin, please visit: 
http://theautomatic.net/yahoo_fin-documentation/

In order to install mysql, please visit:
https://dev.mysql.com/downloads/mysql/
An Oracle user account and password is required to obtain this download. Registration and use of MySQL Community Server are free.

Additionally, install pymysql, from a terminal bash shell:
- pip install pymysql

Once MySQL is installed:
- In a terminal window, create an alias for MySQL
    - alias mysql=/usr/local/mysql/bin/mysql
    - set +H
- In a terminal window, log in to MySQL as root
    - mysql --user=root --password=<root_password>
- Once logged into MySQL: 
    - Create a database named FinanceYahoo
        CREATE DATABASE FinanceYahoo;
    - Create a user named FinanceYahoo
        CREATE USER 'FinanceYahoo'@'localhost' IDENTIFIED BY 'F!n@nc3Y@h00';
    - Grant all permission to user FinanceYahoo on database FinanceYahoo
        GRANT ALL PRIVILEGES ON FinanceYahoo.* TO 'FinanceYahoo'@'localhost';

### Import Libraries from yahoo_fin.stock_info and yahoo_fin.options
Once yahoo_fin is installed, import required libraries as indicated below:

In [ ]:
from yahoo_fin.stock_info import tickers_dow
from yahoo_fin.stock_info import tickers_nasdaq
from yahoo_fin.stock_info import tickers_other
from yahoo_fin.stock_info import tickers_sp500
from yahoo_fin.options import get_expiration_dates
from yahoo_fin.options import get_options_chain

### Import Libraries to interact with MySQL

In [ ]:
from sqlalchemy import create_engine
import pymysql

#This is needed in order to pass special characters in the MySQL connection string
from urllib import parse as urlparse

### Import DateTime

In [ ]:
# This is needed to convert string date time stamps into datetime objects
from dateutil.parser import parse
# This is needed to invoke the pandas to_datetime function
import pandas as pd

## Get the list of unique tickers
- Pull all available tickers
- Eliminate duplicates if they exist
- Eliminate tickers that contain special characters

In [ ]:
tickers = []
tickers = tickers_dow()
tickers.extend(tickers_nasdaq())
tickers.extend(tickers_other())
tickers.extend(tickers_sp500())
tickers = list(dict.fromkeys(tickers)) #This instruction eliminates duplicate tickers
tickers = [item for item in tickers if "-" not in item and "$" not in item and "." not in item]

## Connect to MySQL

In [ ]:
#urllib.parse.quote_plus is used to url encode the special characters in the password
sql_connection_string = 'mysql+pymysql://FinanceYahoo:%s@localhost/FinanceYahoo' % urlparse.quote_plus('F!n@nc3Y@h00')
sql_engine = create_engine(sql_connection_string)
sql_connection = sql_engine.connect()

## Parse Finance Yahoo Data


- Create a function to parse the dataframes resulting from calling get_options_chain

In [ ]:
def parse_dataframe(df,expiration):
    df.loc[df['Last Price'] == '-','Last Price'] = None
    df.loc[df['Bid'] == '-','Bid'] = None
    df.loc[df['Ask'] == '-','Ask'] = None
    df.loc[df['Change'] == '-','Change'] = None
    df['% Change'] = df['% Change'].map(lambda x: x.lstrip('+-').rstrip('%').replace(',',''))
    df.loc[df['% Change'] == '','% Change'] = '0'
    df.loc[df['Open Interest'] == '-','Open Interest'] = '0'
    df.loc[df['Volume'] == '-','Volume'] = '0'
    df['Implied Volatility'] = df['Implied Volatility'].map(lambda x: x.rstrip('%').replace(',',''))
    df['Last Trade Date'] = df['Last Trade Date'].map(lambda x: x.rstrip(' EDT'))
    df['Last Trade Date'] = pd.to_datetime(df['Last Trade Date'],format="%Y-%m-%d %I:%M%p")
    df['Ticker'] = df['Contract Name'].str.extract(r'([A-Z]*)')
    df['Expiration'] = parse(expiration)
    df['OptionType'] = df['Contract Name'].str.extract(r'[A-Z]*[0-9]{6}([A-Z])')
    return df.astype({"% Change" : float, "Volume" : int, "Implied Volatility" : float}) 

### The next code cell is a test showing the result of calling parse_dataframe with the calls dataframe of a specific ticker and expiration date
- This cell does not need to be run except for testing purposes

In [ ]:
#Test
ticker = 'AAPL'
date = get_expiration_dates(ticker)[1]
options_chain = get_options_chain(ticker,date)
dataframe = parse_dataframe(options_chain['calls'],date)
dataframe


### Traverse the list of unique tickers
    - For optionable tickers, get all available expiration dates
    - Get the option chains (calls and puts) for each ticker / expiration date pair
    - Append each option to the database

In [ ]:
ticker_count = 1

for ticker in tickers:
    print(f'{ticker}: {ticker_count} of {len(tickers)}')    
    ticker_count += 1
    
    for date in get_expiration_dates(ticker):
        try:
            options_chain = get_options_chain(ticker,date)
        except:
            print(f'No option chains for {ticker},{date}')
        
        try:
            df = parse_dataframe(options_chain['calls'],date)
            df.to_sql('Option',sql_connection,if_exists='append')
        except:
            print(f'Skipped {ticker},{date}')
        
        try:
            df = parse_dataframe(options_chain['puts'],date)
            df.to_sql('Option',sql_connection,if_exists='append')
        except:
            print(f'Skipped {ticker},{date}')
print('Done')